# SUICIDES RATE ANALYSIS

###  import module

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

###  explore data

In [ ]:
df = pd.read_csv('./doc/master.csv')

In [ ]:
list(df)

untuk mempermudah, dilakukan perubahan nama kolom dari data frame

In [ ]:
# Rename nama kolom
df.rename(columns={'suicides/100k pop':'suicides_per_100k_pop',
                  'country-year': 'country_year',
                  'HDI for year':'HDI',
                  ' gdp_for_year ($) ':'gdp_for_year',
                  'gdp_per_capita ($)':'gdp_percapita'}, inplace=True)

In [ ]:
# data info
df.info()

In [ ]:
df['gdp_for_year'] = df.gdp_for_year.apply(lambda x:x.replace(",","")).astype(np.int64)

In [ ]:
df.sample(5)

### Analisis Statistik
#### GDP dan Suicide
GDP (PDB / Pendapatan Domestik Bruto) adalah variabel yang menjadi perhatian di beberapa negara yang mungkin merupakan salah satu faktor yang menjadi penyebab meningkatnya rasio bunuh diri, oleh karena itu akan di;akukam test untuk melihat hubungan antara GDP dan Jumlah Suicide.

Pertama2 yang dilakukan adalah melihat korelasi antara "suicide_no" dan "gdp_for_year".

In [ ]:
df.suicides_no.describe()

In [ ]:
df.suicides_no.plot(grid=True,kind='hist',figsize=(15,8))
plt.xlabel('suicides_no')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Calc Skewness
df.suicides_no.skew()

nilai skewness dari variable "suicides_no" terlalu tinggi (10.35). maka tidak bisa dilakukan test korelasi menggunakan metode spearman, karena untuk menghitung korelasi menggunakan metode Pearson maka variabelnya harus berdistribusi normal. Cara termudahnya adalah dengan melihat nilai skewness dari variabel tersebut, nilai skewnwss tidak boleh lebih besar dari 1 (Skewness < 1), solusi lainnya selain melakukan transformasi adalah denan menggunaka metode selain Pearson (Kendall atau Spearman).

In [ ]:
df.gdp_for_year.describe()

In [ ]:
df.gdp_for_year.plot(grid=True,kind='hist',figsize=(15,8))
plt.xlabel('gdp_for_year')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df.gdp_for_year.skew()

terlihat, kedua variabel tidak berdistribusi normal (Skewness > 1). maka digunakan metode spearman untuk melihat korelasi antara dua variabel

In [ ]:
df['suicides_no'].corr(df['gdp_for_year'],method='spearman')

Terdapat korelasi yagn signifikan antara variabel dependent "Suicides_no" adan "gdp_for_year". namun belum cukup. nilai korelasi hanya ~ 0.66 (0.8 dipandang cukup). langkah selanjutnya adalah dengan melihat confidence interval untuk melihat adakah korelasi pada populasi

In [ ]:
r_z = np.arctanh(df['suicides_no'].corr(df['gdp_for_year'],method='spearman'))
se = 1/np.sqrt(df.suicides_no.count()-3)
alpha = 0.05
z = stats.norm.ppf(1-alpha/2)
lo_z, hi_z = r_z-z*se, r_z+z*se
lo, hi = np.tanh((lo_z, hi_z))

In [ ]:
# Confidence Interval
lo,df['suicides_no'].corr(df['gdp_for_year'],method='spearman'),hi

Terdapat korelasi diantarakedua variable ini di populasi, dan berada duantara ~0.65 dan ~0.66, secara statistik interval ini tergolong kecil

#### HDI(Human Development Index) and Suicide

Seperti yang sebelumnya, perlunya melihat nilai skewness untuk memilih metode yang tepat untuk dilakukannya test.

In [ ]:
#Checking the HDI assumption
df.HDI.describe(), "Null = %a" % df.HDI.isna().sum(), "skew = %s" %df.HDI.skew()

In [ ]:
df.HDI.plot(kind='hist', bins=30, grid=True, figsize=(15,8))
plt.xlabel('HDI for year')
plt.ylabel('Frequency')
plt.show()

NIlai Skewness HDI < 1, maka bisa dikatakan variabel ini berdistribusi normal, walaupun "BellCurve" nya tak sempurna, namun dapat diterima.

Walaupun variabel ini berdistribusi normal, namun belum dapat dilakukan test menggunakan metode Pearson, karena variabel lainnya (suicides_no) yang akan kita cari korelasinya tidaklah berdistribusi normal, maka akan dilakukan tranformasi pada distribusi tersebut sebelum dilakukan test.

In [ ]:
# Transforming suicides_no variable
df['suicides_no_log'] = df['suicides_no'].apply(lambda x: math.log(x+1))
df.suicides_no_log.skew()

In [ ]:
df.suicides_no_log.plot(kind='hist', bins=30, grid=True, figsize=(15,8))
plt.xlabel('suicides_no_log')
plt.ylabel('Frequency')
plt.show()

bentuk "Bell-curve"nya tak sempurna, namun cukuplah...

In [ ]:
# finding the correlation
df.suicides_no_log.corr(df.HDI)

In [ ]:
r_z = np.arctanh(df.suicides_no_log.corr(df.HDI))
se = 1/np.sqrt(df.suicides_no_log.count()-3)
alpha = 0.05
z = stats.norm.ppf(1-alpha/2)
lo_z, hi_z = r_z-z*se, r_z+z*se
lo, hi = np.tanh((lo_z, hi_z))

In [ ]:
# Confidence Interval
lo,df.suicides_no_log.corr(df.HDI),hi

Terdapat korelasi yang signifikan antara variabel dependent (suicides_no) dengan variabel independent (HDI). tapi nilai korelasinya relatif kecil. hanya  ~0.18. bahkan untuk nlai Populasi berada diantara ~ 0.16 dan ~0.19

## Regression Analysis
sebelumnya telah dilakukn dua analisis korelasi, yang dibahas hanya hubungan antara dua variabel dan bukan penyebabnya. Belum diketahui bahwa variabel yang satu mempengaruhi variabel yang lainnya, contohnya hanya terdapat nilai korelasi positif yang sedikit antara HDI dan suicides rate. Korelasi hanya memberi informasi tentang hubungan antara dua variabel. maka kali ini akan dilakukan anallisis regresi untuk mencari tahu variabel yang mempengaruhi suicide rate

untuk melakukan analisis regresi, variabel harus berditribusi normal, maka akan kita list variabel dan akan dilakukan transformasikan variabel tersebut apabila diperlukan

In [ ]:
df.describe(include='all')

sebelum dilakukan transformasi

In [ ]:
w=15
h=8
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
df.gdp_for_year.plot(kind='hist', ax=ax1,figsize=(w,h))
df.population.plot(kind='hist', ax=ax2,figsize=(w,h))
df.gdp_percapita.plot(kind='hist', ax=ax3,figsize=(w,h))
df.suicides_per_100k_pop.plot(kind='hist', ax=ax4,figsize=(w,h))


terlihat bahwa kempat variabel diatas tidak berdistribusi normal, maka akan dilakukan tranformasi variabel tersebut dengan logaritma natural

In [ ]:
# Transformasi variable
df['gdp_for_year_log'] = df['gdp_for_year'].apply(lambda x: math.log(x+1))
df['population_log'] = df['population'].apply(lambda x: math.log(x+1))
df['gdp_percapita_log'] = df['gdp_percapita'].apply(lambda x: math.log(x+1))
df['suicides_per_100k_pop_log'] = df['suicides_per_100k_pop'].apply(lambda x: math.log(x+1))

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
df.gdp_for_year_log.plot(kind='hist', ax=ax1,figsize=(10,7),bins=20)
df.population_log.plot(kind='hist', ax=ax2,figsize=(10,7),bins=20)
df.gdp_percapita_log.plot(kind='hist', ax=ax3,figsize=(10,7),bins=20)
df.suicides_per_100k_pop_log.plot(kind='hist', ax=ax4,figsize=(10,7),bins=20)

tidak terbentuk "Bell curve" yang sempurma, tapi ini cukup.
kita akan melihat variasi antara variabel dependen dan variabel independent/predictor, dengan cara memasukkan variabel independen satu per satu dengan menggunakan funsi ols (ordinary least squares) yang terdapat pada module stats model

In [ ]:
model1 = smf.ols('suicides_no_log ~ HDI', data = df).fit()
# model1.rsquared,model1.rsquared_adj
print(model1.summary())

terlihat, HDI for year secara berkontribusi menjadi penyebab dari suicides rate, walaupun nilai Adjusted R-squared nya hanya 0.032.

In [ ]:
model2 = smf.ols('suicides_no_log ~ HDI+gdp_for_year_log', data = df).fit()
# model2.rsquared,model2.rsquared_adj
print(model2.summary())

ketika ditambahkan variabel lain (gdp_for_year_log), nilai dari adjusted R-squarred meningkat dari 0.032 menjadi 0.474.

In [ ]:
model3 = smf.ols('suicides_no_log ~ HDI+gdp_for_year_log+population_log', data=df).fit()
# model3.rsquared,model3.rsquared_adj
print(model3.summary())

nilai dari adjusted R-squarred meningkat dari 0474 menjadi 0.594, menunjukkan bahwa variabel populastion_log memberikan kontribusi dalam meningkatkan varias adjusted R-squarred

In [ ]:
model4 = smf.ols('suicides_no_log ~ HDI+gdp_for_year_log+population_log+gdp_percapita_log',data=df).fit()
# model4.rsquared,model4.rsquared_adj
print(model4.summary())

In [ ]:
model5 = smf.ols('suicides_no_log ~ HDI+gdp_for_year_log+population_log+gdp_percapita_log+suicides_per_100k_pop_log',data=df).fit()
# model5.rsquared,model5.rsquared_adj
print(model5.summary())

terlihat lonjakan besar terjadi pada  nilai adjusted R-squarred setelah menambahkn variabel suicides_per_100k_pop_log menjadi 0.917. maka dapat dikatakan bahwa keempat variabel independent diatas () mbisa menjadi representasi dari penyebab suicide rate 91.7%.

untuk mengevaluasi model diatas, maka distribusi dari residu model diatas haruslah berdistribusi normal.

In [ ]:
model5.resid.plot(kind='hist',bins=71,figsize=(15,8))

### Kesimpulan
Setelah dilakukan Anallisis regresi, diperolehlah penyebab dari suicide rate dari dataset ini. terlihat semakin kecil GDP per capita maka semakin besar suicide rate, begitu pula pengaruh variabel independen lainnya dengan suicide rate.